# TimeGAN Tutorial

## Time-series Generative Adversarial Networks

- Paper: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar, "Time-series Generative Adversarial Networks," Neural Information Processing Systems (NeurIPS), 2019.

- Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks

- Last updated Date: April 24th 2020

- Code author: Jinsung Yoon (jsyoon0823@gmail.com)

This notebook describes the user-guide of a time-series synthetic data generation application using timeGAN framework. We use Stock, Energy, and Sine dataset as examples.

### Prerequisite
Clone https://github.com/jsyoon0823/timeGAN.git to the current directory.

## Necessary packages and functions call

- timegan: Synthetic time-series data generation module
- data_loading: 2 real datasets and 1 synthetic datasets loading and preprocessing
- metrics: 
    - discriminative_metrics: classify real data from synthetic data
    - predictive_metrics: train on synthetic, test on real
    - visualization: PCA and tSNE analyses

In [1]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import wandb
wandb.login()
# 1. TimeGAN model
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: steliobompai (synthetic-data-gan). Use `wandb login --relogin` to force relogin


# Training pipeline for TimeGAN with W&B logging

In [2]:
def training_TimeGAN(config):

    parameters = {
        'data_name': config.data_name,
        'seq_len': config.seq_len,
        'batch_size': config.batch_size,
        'module': config.module,
        'hidden_dim': config.hidden_dim,
        'num_layer': config.num_layer,
        'iterations': config.iterations,
        'embed_iterr': config.embed_iterr,
        'num_train_G': config.num_train_G
    }
    if config.data_name in ['stock', 'energy', 'halcor']:
        ori_data = real_data_loading(parameters['data_name'], parameters['seq_len'])
    print(parameters['data_name'] + ' dataset is ready.')

    generated_data = timegan(ori_data, parameters)   
    print('Finish Synthetic Data Generation')

    # Visualization
    metric_iteration = 5
    discriminative_score = list()
    for _ in range(metric_iteration):
        temp_disc = discriminative_score_metrics(ori_data, generated_data)
        discriminative_score.append(temp_disc)


    predictive_score = list()
    for tt in range(metric_iteration):
        temp_pred = predictive_score_metrics(ori_data, generated_data)
        predictive_score.append(temp_pred)
        
    wandb.log(
        {'Discriminative score': np.round(np.mean(discriminative_score), 4),
         'Predictive score': np.round(np.mean(predictive_score), 4)
        })
    
    f_pca = visualization(ori_data, generated_data, 'pca')
    f_tsne = visualization(ori_data, generated_data, 'tsne')

    wandb.log({
        "PCA Plot": wandb.Image(f_pca),
        "t-SNE Plot": wandb.Image(f_tsne)
    })

    plt.close(f_pca)
    plt.close(f_tsne)


In [3]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'data_name': {
            'values': ['energy']
        },
        'seq_len': {
            'values': [24]
        },
        'batch_size': {
            'values': [128]
        },
        'module' :{
            'values': ['gru']
        },
        'hidden_dim': {
            'values': [28*4] # feature dimension * 4
        },
        'num_layer': {
            'values': [3]
        },
        'iterations': {
            'values': [50000]
        },
        'embed_iterr': {
            'values': [10000]
        },
        'num_train_G': {
            'values': [2]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="TimeGAN_Experiments")

def sweep_train():
    run = wandb.init()
    run.name = (
        f"{run.config.data_name}-"
        f"seq_len-{run.config.seq_len}-"
        f"batch_size-{run.config.batch_size}-"
        f"module-{run.config.module}-"
        f"hidden_dim-{run.config.hidden_dim}-"
        f"num_layer-{run.config.num_layer}-"
        f"iterations-{run.config.iterations}-"
        f"embed_iterr-{run.config.embed_iterr}-"
        f"num_train_G-{run.config.num_train_G}"
    )
    run.save()

    training_TimeGAN(run.config)

wandb.agent(sweep_id, function=sweep_train)

Create sweep with ID: g2o9cbdw
Sweep URL: https://wandb.ai/synthetic-data-gan/TimeGAN_Experiments/sweeps/g2o9cbdw


wandb: Agent Starting Run: hdwfc4qo with config:
wandb: 	batch_size: 128
wandb: 	data_name: energy
wandb: 	embed_iterr: 10000
wandb: 	hidden_dim: 112
wandb: 	iterations: 50000
wandb: 	module: gru
wandb: 	num_layer: 3
wandb: 	num_train_G: 2
wandb: 	seq_len: 24


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


energy dataset is ready.




Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://gith

2025-06-30 03:52:43.671380: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-06-30 03:52:43.675947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-06-30 03:52:43.676035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3080 major: 8 minor: 6 memoryClockRate(GHz): 1.8
pciBusID: 0000:01:00.0
2025-06-30 03:52:43.676083: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2025-06-30 03:52:43.676117: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared obj

...

2025-06-30 07:22:38.264554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:22:38.264576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:22:44.833310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:22:44.833330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:22:51.330237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:22:51.330259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:22:58.148964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:22:58.148984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:23:04.746866: I tensorflow

2025-06-30 07:23:11.206051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:23:11.206071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:23:27.304039: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:23:27.304058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:23:43.340711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:23:43.340731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:23:59.392362: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2025-06-30 07:23:59.392381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      
2025-06-30 07:24:15.404487: I tensorflow

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2000 samples in 0.000s...
[t-SNE] Computed neighbors for 2000 samples in 0.057s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2000
[t-SNE] Computed conditional probabilities for sample 2000 / 2000
[t-SNE] Mean sigma: 0.004626
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.023849
[t-SNE] KL divergence after 300 iterations: 0.914130


Discriminative score,▁
Predictive score,▁
Discriminative score,0.4924
Predictive score,0.3146


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
